# Exercise 2 | TKO_2096 Application of Data Analysis 2021

#### Prediction of the metal ion content from multi-parameter data <br>
- Use K-Nearest Neighbor Regression with euclidean distance to predict total metal concentration (c_total), concentration of Cadmium (Cd) and concentration of Lead (Pb), for each sample using number of neighbors k = 3.<br> <br>

    - You may use Nearest Neighbor Regression from https://scikit-learn.org/stable/modules/neighbors.html
    - The data should be standarized using z-score.
    - Implement your own Leave-One-Out cross-validation and calculate the C-index for each output (c_total, Cd, Pb). 
    - Implement your own Leave-Replicas-Out cross-validation and calculate the C-index for each output (c_total, Cd, Pb).
    - Return your solution as a Jupyter Notebook file (include your full name in the file name).
    - Submit to moodle your solution on ** Wednesday 10 of February** at the latest.

## Import libraries

In [1]:
#In this cell import all libraries you need. For example: 
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
# for comparing the result of c-index, commented out for pdf
#!pip install lifelines
#from lifelines.utils import concordance_index as c_index

## Read and visualize the dataset

In [2]:
#In this cell read the file Water_data.csv
#Print the dataset dimesions (i.e. number of rows and columns)
#Print the first 5 rows of the dataset

df = pd.read_csv('Water_data.csv')
print(df.shape)
print(df[:5])

(225, 6)
   c_total     Cd      Pb    Mod1  Mod2    Mod3
0     2000  800.0  1200.0  126430  2604    6996
1       35   14.0    21.0   20597   271  138677
2       35   14.0    21.0   24566   269  161573
3       35   35.0     0.0  105732   971  132590
4      100   20.0    80.0   57774  5416   93798


#### To show understanding of the data, answer the following questions:
- How many different mixtures of Cadmium (Cd) and Lead (Pb) were measured? <br>
- How many total concentrations (c_total) were measured? <br>
- How many mixtures have less than 4 replicas? <br>
- How many mixtures have 4 or more replicas? Print out c_total, Cd and Pb for those concentrations.<br>

In [3]:
#In this cell write the code to answer the previous questions and print the answers.
# check the amount of unique mixtures
print(df[['Cd', 'Pb']].drop_duplicates().shape[0], 'different mixtures of Cadmium and Lead')
print(df.shape[0], 'total number of concentrations')
print(df['c_total'].drop_duplicates().shape[0], 'different concentrations')

# let's form a dataframe of the different mixtures and add a column to represent the amount of replicas for each
replicas = df.groupby(df.columns[0:3].tolist()).size().reset_index().rename(columns={0:'replicas'})
print(replicas[replicas['replicas'] < 4].shape[0], 'mixtures with less than 4 replicas')
print(replicas[replicas['replicas'] >= 4].shape[0], 'mixtures with 4 or more replicas')
print(replicas[replicas['replicas'] >= 4])


67 different mixtures of Cadmium and Lead
225 total number of concentrations
12 different concentrations
43 mixtures with less than 4 replicas
24 mixtures with 4 or more replicas
    c_total     Cd     Pb  replicas
19       50    0.0   50.0         4
20       50   10.0   40.0         4
21       50   20.0   30.0         4
22       50   30.0   20.0         4
23       50   40.0   10.0         4
24       50   50.0    0.0         4
25       70    0.0   70.0         4
26       70   14.0   56.0         4
27       70   28.0   42.0         4
28       70   42.0   28.0         4
29       70   56.0   14.0         4
30       70   70.0    0.0         4
31      100    0.0  100.0         4
32      100   20.0   80.0         4
33      100   40.0   60.0         4
34      100   60.0   40.0         4
35      100   80.0   20.0         4
36      100  100.0    0.0         4
37      200    0.0  200.0         4
38      200   40.0  160.0         4
39      200   80.0  120.0         4
40      200  120.0   80.0    

## Standardization of the dataset

In [4]:
#Standardize the dataset features by removing the mean and scaling to unit variance. 
#In other words, use z-score to scale the dataset features (Mod1, Mod2, Mod3) 
#Print the 5 first samples (i.e. rows) of the scaled dataset

df_std = df.copy()
for column in df_std.columns[3:]:
    df_std[column] = (df_std[column] - df_std[column].mean()) / df_std[column].std()
print(df_std[:5])

   c_total     Cd      Pb      Mod1      Mod2      Mod3
0     2000  800.0  1200.0  0.166135 -0.507624 -1.495707
1       35   14.0    21.0 -0.890630 -0.700080  0.684335
2       35   14.0    21.0 -0.850999 -0.700245  1.063389
3       35   35.0     0.0 -0.040539 -0.642335  0.583562
4      100   20.0    80.0 -0.519410 -0.275654 -0.058658


## C-index code 

In [9]:
def cindex(true_labels, pred_labels):
    """Returns C-index between true labels and predicted labels
    """  
    n = 0
    pairs = 0
    for i in range(len(pred_labels)):
        for j in range(i+1,len(pred_labels)): # start from i+1 to exclude already visited values
            if true_labels[i] != true_labels[j]: # if pairs are comparable (not equal), add to pairs
                pairs += 1
            if (pred_labels[i] < pred_labels[j] and true_labels[i] < true_labels[j]) or (pred_labels[i] > pred_labels[j] and true_labels[i] > true_labels[j]):
                n = n+1
            if pred_labels[i] == pred_labels[j]:
                n = n+0.5
    cindx = n / pairs
    return cindx

In [10]:
#test cindex function with following values
#from lifelines.utils import concordance_index as c_index
true_labels = [-1, 1, 1, -1, 1]
predictions = [0.60, 0.80, 0.75, 0.75, 0.70]
cindx = cindex(true_labels, predictions)
print(cindx)
# comparison
#print(c_index(true_labels, predictions)) 

0.75


## Functions

In [16]:
#Include here all the functions that you need to run in the data analysis part.

def split_data(df): # split data into folds with replicas
# a nice spaghetti implementation
    df_copy = df.sort_values(by=['c_total', 'Cd', 'Pb']) # sort the dataframe so that the replicas follow each other
    X = df_copy[['Mod1', 'Mod2', 'Mod3']].values.tolist()
    y = df_copy[['c_total', 'Cd', 'Pb']].values.tolist()
    X_folds = []
    y_folds = []
    i = 0
    while len(X_folds) < 67: #we know that there are 67 unique mixtures, so the amount of folds is 67
        X_fold = [X[i]]
        y_fold = [y[i]]
        for j in range(i+1,len(X)): # skip comparing the row with itself
            if y[i][0]== y[j][0] and y[i][1] == y[j][1] and y[i][2] == y[j][2]:
                X_fold.append(X[j]) # if the values are the same in both rows, append to fold
                y_fold.append(y[j])
            else:
                break # since the arrays are sorted, whenever we meet a different value, break the loop
        X_folds.append(X_fold) # append the fold made in the inner loop to the folds
        y_folds.append(y_fold)
        i = j # the next fold starts where the previous ended
    X_folds, y_folds = shuffle(X_folds, y_folds) # shuffle just in case
    X_folds = np.asarray(X_folds, dtype=object)
    y_fold = np.asarray(y_folds, dtype=object)
    return X_folds, y_folds # returns folds

def loo(df): # leave-one-out CV function
    X = df[['Mod1', 'Mod2', 'Mod3']].values
    y = df[['c_total', 'Cd', 'Pb']].values
    X, y = shuffle(X, y) # this might not be necessary, let's do it anyways
    true = []
    preds = []
    for i in range(len(X)):
        X_test = X[i].reshape(1, -1) # take the ith sample as a test set and delete it from the rest -> train
        X_train = np.delete(X,i, 0)
        y_test = y[i].reshape(1, -1)
        y_train = np.delete(y,i, 0)
        knn = KNeighborsRegressor(n_neighbors=3, metric='euclidean')
        knn.fit(X_train, y_train)
        pred = knn.predict(X_test)
        preds.extend(pred)
        true.extend(y_test)
    preds = np.array(preds)
    true = np.array(true)
    print("C-index for c_total:", cindex(true[:,0],preds[:,0]))
    print("C-index for Cd:", cindex(true[:,1],preds[:,1]))
    print("C-index for Pb:", cindex(true[:,2],preds[:,2]))

    
def leave_replicas_out(df): # leave-replicas-out CV function
    true = []
    preds = []
    cindexs = []
    X_folds, y_folds = split_data(df) # splitting the data into folds
    for i in range(len(X_folds)):
        X_test = X_folds[i] # take the ith fold as a test fold and delete it from the rest -> train
        y_test = y_folds[i]
        X_train = np.delete(X_folds, i, 0)
        y_train = np.delete(y_folds, i, 0)
        X_train = np.concatenate([x for x in X_train]) # concatenate into 2D array to evaluate
        y_train = np.concatenate([y for y in y_train])
        knn = KNeighborsRegressor(n_neighbors=3, metric='euclidean')
        knn.fit(X_train, y_train)
        pred = knn.predict(X_test)
        true.extend(y_folds[i])
        preds.extend(pred) # add the predictions and true values to an array
    true = np.array(true)
    preds = np.array(preds)
    #not sure if the c-index should've been calculated and averaged fold-wise, here it's for each prediction
    print("C-index for c_total:",cindex(true[:,0],preds[:,0]))
    print("C-index for Cb:",cindex(true[:,1],preds[:,1]))
    print("C-index for Pb:",cindex(true[:,2],preds[:,2]))

## Results for Leave-One-Out cross-validation

In [12]:
#In this cell run your code for leave-One-Out cross-validation and print the corresponding results.
loo(df_std)

C-index for c_total: 0.9298931456867344
C-index for Cd: 0.9046227084812294
C-index for Pb: 0.8786069236230007


## Results for Leave-Replicas-Out cross-validation

In [18]:
#In this cell run your script for leave-Replicas-Out cross-validation and print the corresponding results.
leave_replicas_out(df_std)

C-index for c_total: 0.8283381113717314
C-index for Cb: 0.7649183613813839
C-index for Pb: 0.7716415417380048


## Interpretation of results
#### Answer the following questions based on the results obtained
- Which cross-validation approach had more optimistic results?
- Which cross-validation generalize better on unseen data? Why?

### In this cell write your answers to the questions about Interpretation of Results.

The leave-one-out naturally produced more optimistic results, since the training data contains replicas of the true values.

The leave-replicas-out probably generalizes better on unseen data, because it hasn't accumulated any bias towards the replicas unlike the leave-one-out.




